In [ ]:
#this helps us mount our google drive and navigate to the right folder and access the video, run and you'll be asked to follow a link, there login with the
#right google id and you'll get a long string, copy it and enter it into the box that appears below and press enter
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
% cd My Drive/Final Year Project/ColabFiles

/gdrive/My Drive/Final Year Project/ColabFiles


In [ ]:
% ls

 Subtitle-ing.ipynb  'test video.mp4'


In [ ]:
#!pip install SpeechRecognition moviepy #run only first time.
import speech_recognition as sr 
import moviepy.editor as mp

     |████████████████████████████████| 32.8MB 113kB/s 
Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.0%)737280/45929032 bytes (1.6%)2031616/45929032 bytes (4.4%)3883008/45929032 bytes (8.5%)6537216/45929032 bytes (14.2%)10117120/45929032 bytes (22.0%)13672448/45929032 bytes (29.8%)17481728/45929032 bytes (38.1%)20979712/45929032 bytes (45.7%)24690688/45929032 bytes (53.8%)28409856/45929032 bytes (61.9%)32137216/45929032 bytes (70.0%)35831808/4592

In [ ]:
#read our video, get the audio
clip = mp.VideoFileClip(r"test video.mp4") 
 
clip.audio.write_audiofile(r"test_audio.wav")

[MoviePy] Writing audio in test_audio.wav


100%|██████████| 3824/3824 [00:04<00:00, 781.21it/s] 

[MoviePy] Done.


In [ ]:
#To Be Replaced: using SpeechRecognition to query speech to text here
r = sr.Recognizer()
audio = sr.AudioFile("test_audio.wav")
with audio as source:
  audio_file = r.record(source)
result = r.recognize_google(audio_file,)

In [ ]:
# exporting the result this is just transcript no srt
with open('recognized.txt',mode ='w') as file: 
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(result) 
   print("ready!")

ready!


In [ ]:
def gettimestamp(secs):
  hrs=int(secs/3600)
  secs=secs%3600
  min=int(secs/60)
  secs=int(secs%60)
  return '{0}'.format(str(hrs).zfill(2))+':'+'{0}'.format(str(min).zfill(2))+':'+'{0}'.format(str(secs).zfill(2))+",000"

In [ ]:
f = open("recognizedsrtfrmt.txt", "w")

In [ ]:
#write srt as text file
audiolen=clip.duration
offset=0
seq=1
while offset<audiolen:
  len=5 if audiolen-offset>5 else audiolen-offset
  f.write(str(seq)+"\n")
  seq=seq+1
  f.write(gettimestamp(int(offset))+" -->"+" "+gettimestamp(int(offset+len))+"\n")
  with audio as source:
    temp = r.record(source, offset=offset, duration=len)
  try:
    f.write(r.recognize_google(temp)+"\n") #ToDo:replace with offline model
  except:
    f.write("unintelligible\n") #catch all, need to fix error
  offset=offset+len
  f.write("\n")
f.close()

In [ ]:
!apt install ffmpeg
#os.system("ffmpeg -i 'test video.mp4' -i 'try.srt' -c copy -c:s mov_text outfile.mp4")

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


0

In [ ]:
os.system("ffmpeg -i try.srt subtitles.ass")
os.system("ffmpeg -i 'test video.mp4' -vf ass=subtitles.ass mysubtitledmovie.mp4")

0

In [ ]:
#@title TRYING MOVIEPY
#TRYING ON MOVIEPY

In [ ]:
import os.path as op

def annotate(clip, txt, txt_color='red', fontsize=50, font='Xolonium-Bold'):
    """ Writes a text at the bottom of the clip. """
    txtclip = mp.TextClip(txt, fontsize=fontsize, font=font, color=txt_color)
    cvc = mp.CompositeVideoClip([clip, txtclip.set_pos(('center', 'bottom'))])
    return cvc.set_duration(clip.duration)
video = mp.VideoFileClip("test video.mp4")

In [ ]:
audiolen=clip.duration
offset=0
seq=1
tempstr=""
annotated_clips=list()
x=""
while offset<audiolen:
  len=5 if audiolen-offset>5 else audiolen-offset
  tempstr=tempstr+str(seq)+"\n"
  seq=seq+1
  tempstr=tempstr+gettimestamp(int(offset))+" -->"+" "+gettimestamp(int(offset+len))+"\n"
  with audio as source:
    temp = r.record(source, offset=offset, duration=len)
  try:
    tempstr=tempstr+r.recognize_google(temp)+"\n"
    x=r.recognize_google(temp)
  except:
    tempstr=tempstr+"unintelligible\n" #catch all, need to fix error
    x="unintelligiblw"
  offset=offset+len
  tempstr=tempstr+"\n"
  #annotated_clips.append(annotate(video.subclip(int(offset), int(offset+len)), x))
print(tempstr)

OSError: ignored

In [ ]:
!pip install srt #run only once
import srt
subs = list(srt.parse(tempstr))
testsrt=srt.compose(subs)

  Created wheel for srt: filename=srt-3.4.1-cp36-none-any.whl size=20571 sha256=fd857d067099aaacff1a2f04d7b0635ec140857959d030b25647ccaa991f2223
  Stored in directory: /root/.cache/pip/wheels/85/9e/29/17bfa5b6d3ce3c026b6bdf49557a269e0ef869492a1bc700b2
Successfully built srt


In [ ]:
#!apt install imagemagick #run first time only #issue
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip

generator = lambda txt: TextClip(txt, font='Arial', fontsize=16, color='white')
subtitles = SubtitlesClip("try.srt", generator)

video = VideoFileClip("test video.mp4")
result = CompositeVideoClip([video, subtitles.set_pos(('center','bottom'))])

result.write_videofile("out.mp4", fps=video.fps, temp_audiofile="temporary-audio.m4a", remove_temp=True, codec="libx264", audio_codec="aac")

OSError: ignored